In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

## Read Data

In [2]:
df_spencer = pd.read_excel('/Users/apple/Documents/Study/MMA 2022S/841 - Supply Chain/om-0549x.xlsx', sheet_name = 'Exhibit 4 Spencer')

In [3]:
df_william = pd.read_excel('/Users/apple/Documents/Study/MMA 2022S/841 - Supply Chain/om-0549x.xlsx', sheet_name = 'Exhbit 4 Williams')

In [4]:
df_spencer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Miles   70 non-null     int64  
 1   Stops   70 non-null     int64  
 2   Hours   70 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.8 KB


In [5]:
df_william.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Miles   61 non-null     int64  
 1   Stops   61 non-null     int64  
 2   Hours   61 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.6 KB


## Linear Regression

In [6]:
regressor_spencer = LinearRegression()
regressor_spencer.fit(df_spencer.iloc[:,0:2], df_spencer.iloc[:,2:])

LinearRegression()

In [7]:
print(regressor_spencer.intercept_)
print(regressor_spencer.coef_)

[3.00367032]
[[0.01801524 0.30845476]]


In [15]:
r_sq = regressor_spencer.score(df_spencer.iloc[:,0:2], df_spencer.iloc[:,2:])
print('coefficient of determination:', r_sq)

coefficient of determination: 0.8430033515604876


In [8]:
regressor_william = LinearRegression()
regressor_william.fit(df_william.iloc[:,0:2], df_william.iloc[:,2:])

LinearRegression()

In [9]:
print(regressor_william.intercept_)
print(regressor_william.coef_)

[4.44783567]
[[0.01551258 0.33864432]]


In [14]:
r_sq = regressor_william.score(df_william.iloc[:,0:2], df_william.iloc[:,2:])
print('coefficient of determination:', r_sq)

coefficient of determination: 0.6628438912570516


## Testing data

In [10]:
X = pd.DataFrame({'Miles': [200, 250,300, 100], 'Stops': [2,3,4,2]})

In [11]:
X

,Miles,Stops
0,200,2
1,250,3
2,300,4
3,100,2


### Check what time will William or Spencer take for the testing dataset

In [12]:
regressor_william.predict(X)

array([[ 8.22764034],
       [ 9.34191366],
       [10.45618699],
       [ 6.67638232]])

In [13]:
regressor_spencer.predict(X)

array([[7.22362796],
       [8.43284475],
       [9.64206154],
       [5.4221039 ]])

> Now we see that for the same distance and stops, William takes more time than spencer

## Recommendations

> Change the compensation structure to take into consideration the number of miles travelled - regardles -It will remove the aspects that introduce variation in the processes 

In [24]:
import statsmodels.api as sm

In [17]:
df_3 = pd.read_excel('/Users/apple/Documents/Study/MMA 2022S/841 - Supply Chain/om-0549x.xlsx', sheet_name = 'Sheet3')

In [19]:
df_3

,Miles,Stops,Hours
0,264,1,8.00
1,272,4,10.00
2,273,2,8.00
3,276,1,9.67
4,276,2,8.25
...,...,...,...
120,227,4,8.75
121,231,8,11.08
122,240,3,8.00
123,265,4,10.60


In [28]:
regressor_comb = LinearRegression()
regressor_comb.fit(df_3.iloc[:,0:2], df_3.iloc[:,2:])

LinearRegression()

In [27]:
x = df_3.iloc[:,0:2]
y = df_3.iloc[:,2:]
  
# adding the constant term
x = sm.add_constant(x)
  
# performing the regression
# and fitting the model
result = sm.OLS(y, x).fit()
  
# printing the summary table
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Hours   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                     298.2
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           1.07e-47
Time:                        10:07:16   Log-Likelihood:                -159.72
No. Observations:                 125   AIC:                             325.4
Df Residuals:                     122   BIC:                             333.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.0136      0.250     16.029      0.000       3.518       4.509
Miles          0.0158      0.001     17.946      0.000       0.014       0.018
Stops          0.3101      0.036      8.569      0.000       0.238       0.382
==============================================================================
Omnibus:                        2.076   Durbin-Watson:                   2.070
Prob(Omnibus):                  0.354   Jarque-Bera (JB):                2.120
Skew:                           0.279   Prob(JB):                        0.346
Kurtosis:                       2.690   Cond. No.                         891.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
print(regressor_comb.intercept_)
print(regressor_comb.coef_)

[4.01363474]
[[0.01579591 0.3101221 ]]


In [31]:
exp = regressor_comb.predict(df_spencer.iloc[:,0:2])

In [34]:
diff = df_spencer.iloc[:,2:] - exp

In [38]:
np.mean(diff)

Hours   -0.390154
dtype: float64

In [41]:
exp_w = regressor_comb.predict(df_william.iloc[:,0:2])
diff_w = df_william.iloc[:,2:] - exp_w

In [42]:
np.mean(diff_w)

Hours    0.504471
dtype: float64

In [44]:
import scipy.stats as stats
stats.ttest_ind(diff,diff_w)

Ttest_indResult(statistic=array([-5.41568039]), pvalue=array([2.88187402e-07]))